In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: default.

In [39]:
from aiida.orm import QueryBuilder, RemoteData, RemoteStashFolderData
from aiida import plugins
from aiida.tools.visualization import Graph
from aiida.common import timezone
from datetime import timedelta
import pandas as pd
pd.set_option('display.max_columns',None)

   ## 4. Dataframe form query

In [66]:
def query_to_df(days = 20, 
                sdate = '2021-01-01', 
                edate = '2021-02-01',
                columns = None,
                list_plugins=[flexpart_cosmo,flexpart_ifs]
                ):

    query = QueryBuilder()
    query.append(list_plugins, tag='calc', filters ={'ctime':{'>':  timezone.now() - timedelta(days=days)},'attributes.process_state': 'finished'}, project=['id'])
    query.append(Dict, with_outgoing='calc', project=['attributes','attributes.simulation_date'])
    query.append(RemoteStashFolderData, with_incoming='calc', project=['attributes.target_basepath'])
    dict_=query.dict()
    uuid_list={}

    for item in range(len(dict_)):
            if dict_[item]['Dict_1']['attributes.simulation_date']!=None:
                uuid_list[dict_[item]['calc']['id']]=[dict_[item]['Dict_1']['attributes.simulation_date']]
    for item in range(len(dict_)):  
            if dict_[item]['Dict_1']['attributes']!=None:
                uuid_list[dict_[item]['calc']['id']].append(dict_[item]['Dict_1']['attributes'])
                
    df = pd.DataFrame.from_dict(uuid_list, orient='index')
    for column in df.columns:
        df = pd.concat([df.drop(column, axis=1), df[column].apply(pd.Series)], axis=1)
    df=df.set_index(0)
    df.index=df.index.rename('date')
    df=df[df.index.to_series().between(sdate, edate)]
    if columns!=None:
        df=df[columns]
    
    return df

<b>APPLY FUNCT</b>

In [69]:
flexpart_cosmo=plugins.CalculationFactory('flexpart.cosmo')
flexpart_ifs=plugins.CalculationFactory('flexpart.ifs')

# add column names to add into the dataframe, None for all
columns=['mass_per_release',
         'synchronisation_interval',
         'convection_parametrization',
         'TEST_32'
        ]

plugins_=[flexpart_ifs]

#start and end date of the simulations
sdate = '2021-01-01'
edate = '2022-01-10'

#Filter calcjobs by days in the past
days = 100

df=query_to_df(days,
               sdate,
               edate,
               list_plugins=plugins_,
               #columns=columns
               )
df

,grid_distance_x,grid_distance_y,output_grid_type,latitude_of_output_grid,number_of_grid_points_x,number_of_grid_points_y,longitude_of_output_grid,grid_distance_x,grid_distance_y,output_grid_type,heights_of_levels,latitude_of_output_grid,number_of_grid_points_x,number_of_grid_points_y,longitude_of_output_grid,age_class,linit_cond,age_spectra,particle_dump,release_chunk,simulation_date,calculate_fluxes,release_duration,concentration_output,dumped_particle_data,output_every_seconds,simulation_direction,smaller_than_tl_factor,output_for_each_release,sampling_rate_of_output,synchronisation_interval,cosmo_model_mixing_height,domain_filling_trajectory,convection_parametrization,concentration_units_at_source,vertical_motion_time_decrease,time_average_of_output_seconds,concentration_units_at_receptor,cosmo_grid_relaxation_zone_width,particle_splitting_time_constant,quasilagrangian_mode_to_track_particles,subgrid_terrain_effect_parameterization,TEST_32,TEST_200,list_of_species,mass_per_release,particles_per_release
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-12-02 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-12-02 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,2,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,2,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,2,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,0,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,1,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,1,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,1,1,4,10800,2,50.0,999999999,False,True,"{'level_type': 1, 'lower_z_level': 32.0, 'uppe...","{'level_type': 1, 'lower_z_level': 200.0, 'upp...",[24],[1.0],50000
2021-01-01 00:00:00,0.02,0.015,0.0,45.48,305.0,205.0,4.96,0.16,0.12,0,"[50.0, 100.0, 200.0, 500.0, 15000.0]",36.0,207,179,-12.0,86400,0,True,4,10800,2021-01-01 00:00:00,False,86400,9,False,10800,-1,2.0,True,60,60,False,0,1,1,4,10800,2,50.0,999999999,Fa

In [70]:
#list columns info
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 2021-12-02 00:00:00 to 2021-01-01 00:00:00
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   grid_distance_x                          8 non-null      float64
 1   grid_distance_y                          8 non-null      float64
 2   output_grid_type                         8 non-null      float64
 3   latitude_of_output_grid                  8 non-null      float64
 4   number_of_grid_points_x                  8 non-null      float64
 5   number_of_grid_points_y                  8 non-null      float64
 6   longitude_of_output_grid                 8 non-null      float64
 7   grid_distance_x                          8 non-null      float64
 8   grid_distance_y                          8 non-null      float64
 9   output_grid_type                         8 non-null      int64  
 10  heights_of_levels      

### 4.1 Save to csv file

In [103]:
#df.to_csv('file.csv')